# Pré-processamento
Este notebook é responsável por pré-processar os dados realizando as seguintes tarefas:
- Tratamento de dados ausentes
- Definição de tipos
- Normalização e discretização
- Limpeza de dados (univariado, bivariado e multivariado)

Este notebook usa o 'clean_vehicles.csv' como dataset, que já não tem mais as colunas 'county', 'region_url' e 'image_url'.

In [1]:
from Utils import *
from tqdm import tqdm

DATASET = "../datasets/clean_vehicles.csv"
df = pd.read_csv(DATASET)

## Definição de tipos
Olhando para a tabela que descreve o dataset em '1-initial-analysis.ipynb' e pela análise feita no notebook, os dados foram definidos nos formatos abaixo.

### Colunas categóricos
- type
- region
- transmission
- manufacturer
- model
- condition
- cylinders
- fuel
- title_status
- drive
- size (removido)
- paint_color
- state

### Colunas do tipo inteiro
- id (removida)
- price
- year
- odometer

### Coluna do tipo float
- lat
- long

### Colunas do tipo object
- url
- description
- vin (removida)

## Tratamento de dados ausentes ou nulos
Antes de podermos definir os tipos dos dados vamos primeiro lidar com os dados nulos ou ausentes.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435849 entries, 0 to 435848
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            435849 non-null  int64  
 1   url           435849 non-null  object 
 2   region        435849 non-null  object 
 3   price         435849 non-null  int64  
 4   year          434732 non-null  float64
 5   manufacturer  415102 non-null  object 
 6   model         429650 non-null  object 
 7   condition     249043 non-null  object 
 8   cylinders     269465 non-null  object 
 9   fuel          432858 non-null  object 
 10  odometer      360701 non-null  float64
 11  title_status  434043 non-null  object 
 12  transmission  433703 non-null  object 
 13  vin           239197 non-null  object 
 14  drive         313838 non-null  object 
 15  size          139888 non-null  object 
 16  type          318741 non-null  object 
 17  paint_color   300602 non-null  object 
 18  desc

In [3]:
# Verifica quais dados são nulos.
df.isnull().sum()

id                   0
url                  0
region               0
price                0
year              1117
manufacturer     20747
model             6199
condition       186806
cylinders       166384
fuel              2991
odometer         75148
title_status      1806
transmission      2146
vin             196652
drive           122011
size            295961
type            117108
paint_color     135247
description         27
state                0
lat               8235
long              8235
dtype: int64

Foi notado que as colunas 'vin' (código único do veículo, não será útil) e 'size' ('type' já fornece a mesma informação) apresentam muitos dados nulos, mais de 50% do tamanho do dataset. Elas serão removidas. A coluna 'id' também será removida pois não será útil, é apenas um identificador da oferta no craig list.

In [3]:
df.drop(columns=['vin', 'size', 'id'], inplace=True)
df.columns

Index(['url', 'region', 'price', 'year', 'manufacturer', 'model', 'condition',
       'cylinders', 'fuel', 'odometer', 'title_status', 'transmission',
       'drive', 'type', 'paint_color', 'description', 'state', 'lat', 'long'],
      dtype='object')

### Removendo linhas com muitos valores vazios
Deixa no DataFrame apenas amostras que contém valores não-nulos em pelo 14 colunas (75% do número total de colunas) das 19 colunas. Reduzindo em 9% o tamanho da base.

In [4]:
drop_thresh = int(len(df.columns) * 0.75)
original_size = len(df)
df.dropna(thresh=drop_thresh, inplace=True)
print("Redução de %.2f%% no tamanho da base." % ((1 - (len(df)/original_size)) * 100))

Redução de 8.94% no tamanho da base.


### Tratamento de dados ausentes em Dados do tipo inteiro
Para os dados do tipo inteiro foi verificado que existe dados nulos apenas nas coluna 'year' e 'odometer'. 

#### Coluna 'year'
Poucas amostras possui valor nulo na coluna 'year', representando 0,2% do dataset. Como o ano é um fator importante para a compra e venda de carros, queremos manter essas amostras.

Foi feita uma análise e notou-se que nesses casos o ano é na maioria das vezes os 4 primeiros dígitos da descrição seguidos da marca e modelo do carro. Usamos a função 'get_year_from_description' para realizar esse processo, a definição da função está no Utils.py.

In [5]:
df['year'].isnull().sum()

1070

In [6]:
# mostra os 20 primeiros caracteres da descrição das linhas que possuem a coluna ano nula
for elem in df[df['year'].isnull().values]['description']:
    try:
        print(elem[0:40])
    except:
        print(elem)

2019 *Chrysler* *Pacifica* Touring Plus 
2019 *Jeep* *Grand Cherokee* Laredo SUV 
2018 *Toyota* *Tacoma* TACOMA SR5 V6 DOU
2019 *Ford* *Super Duty F-350 DRW Cab-Ch
2018 *Subaru* *Forester* 2.5i CVT SUV - 
2018 *Volkswagen* *Jetta* 1.4T S Automat
2018 *Cadillac* *CTS-V Sedan* 4dr Sedan 
2019 *Ford* *F-150* Lariat 4x4 4dr Super
2018 *Honda* *Accord Sedan* Sport 2.0T 1
2019 *Kia* *Optima* LX Automatic Sedan -
2020 *Chevrolet* *5500XD LCF Diesel* 550
2019 *Ford* *Super Duty F-350 DRW Cab-Ch
2019 *Chevrolet* *Corvette* 2dr Grand Sp
2018 *Chevrolet* *Tahoe* LT-4X4-22S-NEW 
2018 *Honda* *Pilot* EX-L 2WD SUV - $28,
2018 *Acura* *TLX*  Sedan - $29,975Call 
2019 *Chevrolet* *Silverado 1500 LD* LT 
2019 *Mitsubishi* *Mirage* ES CVT ONLY 3
2019 *Ford* *Super Duty F-350 DRW Cab-Ch
2018 *Subaru* *Legacy* 2.5i Premium Seda
2018 *Hyundai* *Accent* SE Sedan Automat
2018 *Toyota* *Tacoma* TACOMA SR5 V6 DOU
2019 *Ford* *Super Duty F-350 DRW Cab-Ch
2019 *BMW* *X7* xDrive50i Sports Activit
2019 *Ford* *Esc

In [7]:
get_year_from_description(df)

In [8]:
df.dropna(subset=['description'], inplace=True)
df.isnull().sum()

url                  0
region               0
price                0
year                 3
manufacturer     14882
model             3641
condition       148571
cylinders       127597
fuel              2494
odometer         38208
title_status      1410
transmission      1789
drive            83169
type             78665
paint_color      96709
description          0
state                0
lat               7552
long              7552
dtype: int64

In [9]:
df[df['year'].isnull()]

,url,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,description,state,lat,long
15321,https://chattanooga.craigslist.org/cto/d/ten-m...,chattanooga,2700,NaN,NaN,ep liberty,excellent,6 cylinders,gas,208000.0,rebuilt,automatic,rwd,SUV,grey,03 jeep liberty 2 wheel drive v6 runs good col...,tn,35.6279,-84.7542
170030,https://sandiego.craigslist.org/nsd/cto/d/esco...,san diego,5000,NaN,NaN,yota 4x4,excellent,6 cylinders,gas,NaN,clean,automatic,4wd,pickup,white,"92 toyota 4x4 king cab automatic, v6, JBL blu...",ca,33.1039,-117.0770
333060,https://lakeland.craigslist.org/cto/d/babson-p...,lakeland,3800,NaN,NaN,oor,good,8 cylinders,gas,200000.0,clean,automatic,4wd,truck,white,Nice truck 4wD. Everything works new brakes 80...,fl,27.7618,-81.5749


#### Sobram três linhas em que o ano é nulo:

In [10]:
for elem in df[df['year'].isnull()]['description']:
    print(elem)
    print('\n')

03 jeep liberty 2 wheel drive v6 runs good cold air and hot heat all windows work real nice on the inside and out it does have a reconstructed title 423twofivetwosevenzeroninesix


92 toyota 4x4 king cab automatic, v6, JBL  blue tooth stereo, clean title, 3rd owner, no accident, runs great


Nice truck 4wD. Everything works new brakes 80 percent tires 4.6 v8




Pela descrição so consegue-se informação do ano para linha com indice 170030 e 15321, o toyota de ano 1992 e o jeep de 2003, respectivamente.

In [11]:
df.at[170030, 'year'] = 1992
df.at[15321, 'year'] = 2003

As outra linha vamos descartar

In [12]:
df.dropna(subset=['year'], inplace=True)
df.isnull().sum()

url                  0
region               0
price                0
year                 0
manufacturer     14881
model             3641
condition       148571
cylinders       127597
fuel              2494
odometer         38208
title_status      1410
transmission      1789
drive            83169
type             78665
paint_color      96709
description          0
state                0
lat               7552
long              7552
dtype: int64

O ano foi inferido como float64 pelo pandas, verificamos se há ou não casas decimais no com a função 'get_decimal' definida no script Utils.py para cada linha do dataframe:

In [13]:
df['year'].dtype

dtype('float64')

Como a serie retornada é vazia, todos os anos são números inteiros.

In [14]:
teste = df['year'].apply(get_decimal)
teste = teste[teste > 0]
teste

Series([], Name: year, dtype: float64)

#### Coluna 'odometer'
Vamos tentar pegar informações sobre o odômetro na descrição da oferta. 

In [15]:
# quantidade de nulos na coluna odometer
df['odometer'].isnull().sum()

38208

miles_df contém todas entradas com coluna odometer nula e que a descrição contém a alguma informação sobre as milhas do carro

In [16]:
miles_df = df[df['odometer'].isnull() & df['description'].str.contains('[Mm][Ii][Ll][Ee][Ss]*|[Mm][Ii][Ll][Ee][Aa][Gg][Ee]')]
miles_df

,url,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,description,state,lat,long
0,https://mohave.craigslist.org/ctd/d/lake-havas...,mohave county,3495,2012.0,jeep,patriot,like new,4 cylinders,gas,NaN,clean,automatic,NaN,NaN,silver,"THIS 2012 JEEP PATRIOT IS A 4CYL. AC, STEREO, ...",az,34.4554,-114.2690
14,https://mohave.craigslist.org/ctd/d/lake-havas...,mohave county,4995,2009.0,jeep,liberty,like new,6 cylinders,gas,NaN,clean,automatic,NaN,NaN,black,CHECK OUT THIS SUPER CLEAN 2009 JEEP LIBERTY! ...,az,34.5575,-114.3310
22,https://maine.craigslist.org/cto/d/waterville-...,maine,2200,1999.0,gmc,NaN,good,8 cylinders,gas,NaN,clean,automatic,4wd,SUV,black,STICKERED TILL NOVEMBER!!! Runs and drives!!! ...,me,44.5543,-69.6178
84,https://miami.craigslist.org/brw/ctd/d/miami-2...,south florida,17995,2019.0,mini,cooper convertible s,like new,NaN,gas,NaN,clean,automatic,NaN,convertible,grey,"2019 MINI COOPER CONVERTIBLE SPORT, LIKE NEW, ...",fl,25.9630,-80.1702
91,https://miami.craigslist.org/brw/ctd/d/miami-2...,south florida,4995,2009.0,jeep,grand cherokee,like new,NaN,gas,NaN,clean,automatic,NaN,SUV,grey,2009 Jeep Grand Cherokee with only a 111.400 M...,fl,25.9629,-80.1699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435768,https://maine.craigslist.org/cto/d/gorham-2004...,maine,3000,2004.0,subaru,forester,fair,4 cylinders,gas,NaN,clean,manual,NaN,SUV,NaN,motor 60-70k miles just recently have put in. ...,me,43.6843,-70.4680
435791,https://oregoncoast.craigslist.org/ctd/d/eugen...,oregon coast,8990,2009.0,jeep,liberty,excellent,NaN,gas,NaN,clean,automatic,NaN,NaN,NaN,We are open to serve our community! With the...,or,44.0451,-123.0440
435799,https://oregoncoast.craigslist.org/ctd/d/eugen...,oregon coast,10990,2014.0,ford,focus titanium,excellent,NaN,gas,NaN,clean,automatic,NaN,NaN,NaN,We are open to serve our community! With the...,or,44.0451,-123.0440
435804,https://mohave.craigslist.org/cto/d/bullhead-c...,mohave county,6400,2010.0,mini,cooper s,like new,4 cylinders,gas,NaN,clean,automatic,fwd,coupe,white,2010 MINI COOPER S HARDTOP WITH PANORAMIC SUNR...,az,35.0861,-114.6120


In [17]:
# tenta obter informação sobre as milhas de um carro na descrição do anúncio
get_odometer_from_description(df)

Não foi possível encontrar as milhas do carro na descrição em 37 amostras.


In [18]:
df.isnull().sum()

url                  0
region               0
price                0
year                 0
manufacturer     14881
model             3641
condition       148571
cylinders       127597
fuel              2494
odometer         22896
title_status      1410
transmission      1789
drive            83169
type             78665
paint_color      96709
description          0
state                0
lat               7552
long              7552
dtype: int64

In [19]:
# verifica se há amostras do odometro que possam conter casas decimais
teste = df[df['odometer'].notnull()]['odometer'].apply(get_decimal)
teste = teste[teste > 0]
teste

Series([], Name: odometer, dtype: float64)

Para as linhas restantes que ainda possuem o valor 'odometer' nulo, atribuimos o valor de -1.

In [20]:
df['odometer'].fillna(-1, inplace=True)

In [21]:
df['odometer'].isnull().sum()

0

### Tratamento de dados ausentes em Dados categóricos
Para os dados categóricos iremos criar uma nova categoria 'undefined' para substituir os dados ausentes.

In [22]:
categoric_types = ['type', 'region', 'transmission', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'title_status', 'drive', 'paint_color', 'state']

for column in categoric_types:
    df[column] = df[column].fillna('undefined')

In [23]:
# confirmando que não há mais amostras com dados categoricos nulos no novo dataframe
df.isnull().sum()

url                0
region             0
price              0
year               0
manufacturer       0
model              0
condition          0
cylinders          0
fuel               0
odometer           0
title_status       0
transmission       0
drive              0
type               0
paint_color        0
description        0
state              0
lat             7552
long            7552
dtype: int64

### Tratamento de dados ausentes em Dados do tipo object
Como retiramos a coluna 'vin' do dataset, não há mais colunas de tipo object com valores nulos.

In [24]:
df[['url', 'description']].isnull().sum()

url            0
description    0
dtype: int64

### Tratamento de dados ausentes em Dados do tipo Float
Os únicos dados floats nulos no dataset são as coordenadas (latitude e longitude).
#### Latitude e Longitude
Para as colunas de latitude e longitude, decidimos pegar as coordenadas dos estados nas linhas que possuiam latitude ou longitude nulas. Foi usado o módulo 'Nominatin' da biblioteca 'geopy', esse módulo permite obter informações como endereço, latitude, longitude e outros dados de localização a partir de uma string passada contendo o nome do local a ser consultado.

Para nosso projeto listamos todos os 51 estados dos EUA, eles estão definidos em formato de dicionário no arquivo 'Constants.py' na variável STATES_DICT. Usando esse dicionário fizemos chamadas ao módulo 'Nominatin' para obter as coordenadas centrais de cada um dos estados dos Estados Unidos. Finalmente, substituímos os dados nulos de coordenadas no dataframe por esses retornados pelo módulo 'Nominatin'. A função responsável é 'put_coordinates_in_dataframe' e está no 'Utils.py'.

In [3]:
df[['lat', 'long']].isnull().sum()

lat     7552
long    7552
dtype: int64

In [4]:
null_coords_df = df[df['lat'].isnull() & df['long'].isnull()]
null_coords_df

,url,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,description,state,lat,long
278,https://greenville.craigslist.org/ctd/d/bmw-6-...,greenville / upstate,0,2013.0,bmw,640 gran coupe,undefined,6 cylinders,gas,62140.0,clean,automatic,rwd,sedan,grey,BMW 6 Series 640 Gran Coupe Sunroof Leather Po...,sc,NaN,NaN
327,https://bham.craigslist.org/ctd/d/lexus-lx-570...,birmingham,0,2011.0,lexus,lx 570,undefined,8 cylinders,gas,110785.0,clean,automatic,4wd,SUV,silver,Lexus LX 570 Mark Levinson Audio Rear DVD Wide...,al,NaN,NaN
351,https://richmond.craigslist.org/ctd/d/2008-bmw...,richmond,6200,2008.0,bmw,3-series,excellent,6 cylinders,gas,124511.0,clean,automatic,rwd,sedan,silver,2008 *** BMW 3-Series 4dr Sdn 328i RWD Sedan *...,va,NaN,NaN
473,https://bham.craigslist.org/ctd/d/volkswagen-t...,birmingham,0,2015.0,volkswagen,touareg,undefined,6 cylinders,gas,52494.0,clean,automatic,4wd,SUV,grey,Volkswagen Touareg V6 Lux 4Motion Navigation B...,al,NaN,NaN
508,https://bham.craigslist.org/ctd/d/cadillac-elr...,birmingham,0,2014.0,cadillac,elr,undefined,undefined,electric,49333.0,clean,automatic,fwd,coupe,silver,Cadillac ELR Luxury Package Blind Spot Cross T...,al,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396153,https://richmond.craigslist.org/ctd/d/2010-che...,richmond,5700,2010.0,chevrolet,impala,excellent,6 cylinders,gas,102026.0,clean,automatic,fwd,sedan,silver,2010 *** Chevrolet Impala 4dr Sdn LT Sedan ***...,va,NaN,NaN
396294,https://greenville.craigslist.org/ctd/d/acura-...,greenville / upstate,0,2017.0,acura,rdx,undefined,6 cylinders,gas,87014.0,clean,automatic,4wd,SUV,white,Acura RDX Advance Package SH-AWD Navigation Ba...,sc,NaN,NaN
396620,https://greenville.craigslist.org/ctd/d/cadill...,greenville / upstate,0,2016.0,cadillac,escalade,undefined,8 cylinders,gas,87899.0,clean,automatic,4wd,SUV,black,Cadillac Escalade Premium Navigation Backup Ca...,sc,NaN,NaN
396824,https://richmond.craigslist.org/ctd/d/2004-cad...,richmond,5800,2004.0,cadillac,escalade,excellent,8 cylinders,gas,169295.0,clean,automatic,4wd,undefined,white,2004 *** Cadillac Escalade 4dr AWD *** Dri...,va,NaN,NaN


In [6]:
put_coordinates_in_dataframe(df)

In [57]:
df.isnull().sum()

url             0
region          0
price           0
year            0
manufacturer    0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
type            0
paint_color     0
description     0
state           0
lat             0
long            0
dtype: int64

In [7]:
df.to_csv('../datasets/clean_vehicles_2.csv', index=False)

## Setando os tipos do dataframe
Com todos os dados nulos devidamente tratados podemos setar os tipos dos dados no dataframe.

In [8]:
set_df_vehicles_data_types(df)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396901 entries, 0 to 396900
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   url           396901 non-null  object  
 1   region        396901 non-null  category
 2   price         396901 non-null  int64   
 3   year          396901 non-null  int64   
 4   manufacturer  396901 non-null  category
 5   model         396901 non-null  category
 6   condition     396901 non-null  category
 7   cylinders     396901 non-null  category
 8   fuel          396901 non-null  category
 9   odometer      396901 non-null  int64   
 10  title_status  396901 non-null  category
 11  transmission  396901 non-null  category
 12  drive         396901 non-null  category
 13  type          396901 non-null  category
 14  paint_color   396901 non-null  category
 15  description   396901 non-null  object  
 16  state         396901 non-null  category
 17  lat           396901 non-null

AttributeError: 'Index' object has no attribute 'hist'